In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar10-python/cifar-10-python.tar.gz
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_1
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_2
/kaggle/input/cifar10-python/cifar-10-batches-py/batches.meta
/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_3
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_5
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_4
/kaggle/input/cifar10-python/cifar-10-batches-py/readme.html


In [2]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch
import torch.nn as nn
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import torch.optim as optim
from torchvision import models

In [3]:
import pickle
import numpy as np

def unpickle(file):
    with open(file, 'rb') as f:
        dict = pickle.load(f, encoding='bytes')
    return dict

In [4]:
def load_cifar10_data(data_dir):
    train_data = []
    train_labels = []

    # Load training batches
    for i in range(1, 6):
        batch = unpickle(f"{data_dir}/data_batch_{i}")
        train_data.append(batch[b'data'])
        train_labels += batch[b'labels']

    train_data = np.concatenate(train_data, axis=0)
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)  # Reshape to (N, H, W, C)
    train_labels = np.array(train_labels)

    # Load test batch
    test_batch = unpickle(f"{data_dir}/test_batch")
    test_data = test_batch[b'data'].reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_labels = np.array(test_batch[b'labels'])

    return (train_data, train_labels), (test_data, test_labels)


In [5]:


class CIFAR10Dataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # image = self.data[idx]
        image = Image.fromarray(self.data[idx])
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.long)

# Load data
(data_dir) = '/kaggle/input/cifar10-python/cifar-10-batches-py'
(train_data, train_labels), (test_data, test_labels) = load_cifar10_data(data_dir)


In [6]:
# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_transform = transforms.Compose([
    transforms.Resize((224,224)),#usng this for AlexNet
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create PyTorch datasets and loaders
train_dataset = CIFAR10Dataset(train_data, train_labels, transform=test_transform)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
device

'cuda'

In [10]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32,64,3,1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(64,128,3,1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Flatten(),
            nn.Linear(373248,512),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(512,10)
        )

    def forward(self,x):
        x = self.conv_layer(x)
        return x

In [11]:
model = CNN().to(device)

In [ ]:
optimizer = optim.SGD(model.parameters(),lr=1e-02)
loss_fn = nn.CrossEntropyLoss()
epochs = 10

In [ ]:
train_losses,test_losses = [],[]
for epoch in range(epochs):
    model.train()
    train_loss =0
    for batch,(x,y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        pred = model(x)
        loss = loss_fn(pred,y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        #Metrics 
        train_loss += loss.item()
        
    avg_loss = train_loss/len(train_loader)
    train_losses.append(avg_loss)
    print(f'Epoch: {epoch} | Train loss: {avg_loss}')

    model.eval()
    with torch.no_grad():
        test_loss,correct=0,0
        total_samples = 0
        for batch,(x,y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            pred = model(x)
            loss = loss_fn(pred,y)
            total_samples += y.size(0)
            test_loss += loss.item()
            correct += ((torch.argmax(torch.softmax(pred,dim=1),dim=1)==y).sum().item())
        avg_test_loss = test_loss/len(test_loader)
        avg_correct = correct/total_samples
        test_losses.append(avg_test_loss)
        print(f'Test_loss {avg_test_loss} | Accuracy {avg_correct}')

In [ ]:
(torch.argmax(torch.softmax(pred,dim=1),dim=1)==y).sum()

In [ ]:
plt.plot(train_losses,label='train')
plt.plot(test_losses,label='test')
plt.legend()


Alexnet

In [ ]:
from torchvision.models import alexnet

In [ ]:
alex = alexnet(pretrained=True)

In [ ]:
alex.classifier[6] = nn.Linear(4096,10) #Changing this as the out_features for alexnet is 1000

In [ ]:
alex = alex.to(device)

In [ ]:
#hyperparameters
alex_optim = optim.SGD(alex.parameters(),lr = 1e-03)
epochs = 5

In [ ]:
train_losses,test_losses = [],[]
for epoch in range(epochs):
    model.train()
    train_loss =0
    for batch,(x,y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        pred = alex(x)
        loss = loss_fn(pred,y)
        loss.backward()
        alex_optim.step()
        alex_optim.zero_grad()

        #Metrics 
        train_loss += loss.item()
        
    avg_loss = train_loss/len(train_loader)
    train_losses.append(avg_loss)
    print(f'Epoch: {epoch} | Train loss: {avg_loss}')

    model.eval()
    with torch.no_grad():
        test_loss,correct=0,0
        total_samples = 0
        for batch,(x,y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            pred = alex(x)
            loss = loss_fn(pred,y)
            total_samples += y.size(0)
            test_loss += loss.item()
            correct += ((torch.argmax(torch.softmax(pred,dim=1),dim=1)==y).sum().item())
        avg_test_loss = test_loss/len(test_loader)
        avg_correct = correct/total_samples
        test_losses.append(avg_test_loss)
        print(f'Test_loss {avg_test_loss} | Accuracy {avg_correct}')

In [ ]:
plt.plot(train_losses,label='train')
plt.plot(test_losses,label='test')
plt.legend()


RESNET 

In [ ]:
# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_transform = transforms.Compose([
    transforms.Resize((224,224)),#usng this for AlexNet
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create PyTorch datasets and loaders
train_dataset = CIFAR10Dataset(train_data, train_labels, transform=test_transform)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,num_workers = 4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False,num_workers = 4)

In [ ]:
resnetmodel = models.resnet18(pretrained=True)

In [ ]:
for param in resnetmodel.parameters():
    param.requires_grad = False

In [ ]:
resnetmodel.fc.in_features

In [ ]:
#modify the FCN
input_tolinear = resnetmodel.fc.in_features
resnetmodel.fc = nn.Linear(in_features=input_tolinear,out_features = 10)
resnetmodel = resnetmodel.to(device)

In [ ]:
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau


In [ ]:
#hyperparam
resnet_optim = optim.SGD(resnetmodel.parameters(),lr=1e-03,momentum=0.9,weight_decay=0.0001)
# resnet_optim = optim.Adam(resnetmodel.parameters(),lr=1e-03,weight_decay=0.0001)

# scheduler = lr_scheduler.StepLR(resnet_optim, step_size=5, gamma=0.1)
scheduler = ReduceLROnPlateau(resnet_optim, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)

epochs = 50
loss_fn = nn.CrossEntropyLoss()

In [ ]:
train_losses,test_losses = [],[]
for epoch in range(epochs):
    resnetmodel.train()
    train_loss =0.0
    for batch,(x,y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        pred = resnetmodel(x)
        loss = loss_fn(pred,y)
        loss.backward()
        resnet_optim.step()
        resnet_optim.zero_grad()

        #Metrics 
        train_loss += loss.item()
        
    avg_loss = train_loss/len(train_loader)
    train_losses.append(avg_loss)
    print(f'Epoch: {epoch} | Train loss: {avg_loss}')

    resnetmodel.eval()
    with torch.no_grad():
        test_loss,correct=0,0
        total_samples = 0
        for batch,(x,y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            pred = resnetmodel(x)
            loss = loss_fn(pred,y)
            total_samples += y.size(0)
            test_loss += loss.item()
            correct += ((torch.argmax(torch.softmax(pred,dim=1),dim=1)==y).sum().item())
        avg_test_loss = test_loss/len(test_loader)
        avg_correct = correct/total_samples
        test_losses.append(avg_test_loss)
        print(f'Test_loss {avg_test_loss} | Accuracy {avg_correct}')
        scheduler.step(avg_test_loss)


In [ ]:
plt.plot(train_losses,label='train')
plt.plot(test_losses,label='test')
plt.legend()

ResNEXT

In [ ]:
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# Create PyTorch datasets and loaders
train_dataset = CIFAR10Dataset(train_data, train_labels, transform=test_transform)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
resnextmodel = models.resnext50_32x4d(pretrained=True)
resnextmodel = resnextmodel.to(device)

In [ ]:
device = next(resnextmodel.parameters()).device
print(f"The model is on: {device}")

In [ ]:
# Modify the final layer to match the number of classes
num_ftrs = resnextmodel.fc.in_features
resnextmodel.fc = nn.Linear(num_ftrs, 10)

In [ ]:
import torch.optim as optim

resnext_optim = optim.Adam(resnextmodel.parameters(), lr=1e-03, weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()
epochs = 10
from torch.optim import lr_scheduler

scheduler = lr_scheduler.StepLR(resnext_optim, step_size=5, gamma=0.1)

In [ ]:
train_losses,test_losses = [],[]
for epoch in range(epochs):
    resnextmodel.train()
    train_loss =0
    for batch,(x,y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        print(x)
        pred = resnextmodel(x)
        loss = loss_fn(pred,y)
        loss.backward()
        resnext_optim.step()
        resnext_optim.zero_grad()

        #Metrics 
        train_loss += loss.item()
        
    avg_loss = train_loss/len(train_loader)
    train_losses.append(avg_loss)
    print(f'Epoch: {epoch} | Train loss: {avg_loss}')
    scheduler.step()

    resnextmodel.eval()
    with torch.no_grad():
        test_loss,correct=0,0
        total_samples = 0
        for batch,(x,y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            pred = resnextmodel(x)
            loss = loss_fn(pred,y)
            total_samples += y.size(0)
            test_loss += loss.item()
            correct += ((torch.argmax(torch.softmax(pred,dim=1),dim=1)==y).sum().item())
        avg_test_loss = test_loss/len(test_loader)
        avg_correct = correct/total_samples
        test_losses.append(avg_test_loss)
        print(f'Test_loss {avg_test_loss} | Accuracy {avg_correct}')

EfficientNet

In [ ]:
# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_transform = transforms.Compose([
    transforms.Resize((224,224)),#usng this for AlexNet
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create PyTorch datasets and loaders
train_dataset = CIFAR10Dataset(train_data, train_labels, transform=test_transform)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,num_workers = 4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False,num_workers = 4)

In [ ]:
efficientmodel = models.efficientnet_b0(pretrained = True)

In [ ]:
#freezing layers and changing classes
efficientmodel.classifier[1] =  torch.nn.Linear(efficientmodel.classifier[1].in_features, 10)

In [ ]:
efficientmodel = efficientmodel.to(device)

In [ ]:
#hyperparam
eff_optim = optim.SGD(efficientmodel.parameters(),lr=1e-03,momentum=0.9,weight_decay=0.0001)
# resnet_optim = optim.Adam(resnetmodel.parameters(),lr=1e-03,weight_decay=0.0001)

# scheduler = lr_scheduler.StepLR(resnet_optim, step_size=5, gamma=0.1)
# scheduler = ReduceLROnPlateau(resnet_optim, mode='min', factor=0.05, patience=2, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=True)

epochs = 50
loss_fn = nn.CrossEntropyLoss()

In [ ]:
train_losses,test_losses = [],[]
for epoch in range(epochs):
    efficientmodel.train()
    train_loss =0
    for batch,(x,y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        # print(x)
        pred = efficientmodel(x)
        loss = loss_fn(pred,y)
        loss.backward()
        eff_optim.step()
        eff_optim.zero_grad()

        #Metrics 
        train_loss += loss.item()
        
    avg_loss = train_loss/len(train_loader)
    train_losses.append(avg_loss)
    print(f'Epoch: {epoch} | Train loss: {avg_loss}')
    # scheduler.step()

    efficientmodel.eval()
    with torch.no_grad():
        test_loss,correct=0,0
        total_samples = 0
        for batch,(x,y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            pred = efficientmodel(x)
            loss = loss_fn(pred,y)
            total_samples += y.size(0)
            test_loss += loss.item()
            correct += ((torch.argmax(torch.softmax(pred,dim=1),dim=1)==y).sum().item())
        avg_test_loss = test_loss/len(test_loader)
        avg_correct = correct/total_samples
        test_losses.append(avg_test_loss)
        print(f'Test_loss {avg_test_loss} | Accuracy {avg_correct}')

MobileNET

In [14]:
# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_transform = transforms.Compose([
    transforms.Resize((224,224)),#usng this for AlexNet
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create PyTorch datasets and loaders
train_dataset = CIFAR10Dataset(train_data, train_labels, transform=test_transform)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,num_workers = 4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,num_workers = 4)

In [15]:
mobilenet_model = models.mobilenet_v2(pretrained = True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 118MB/s] 


In [16]:
mobilenet_model.classifier[1] = nn.Linear(mobilenet_model.last_channel,10)

In [17]:
mobilenet_model = mobilenet_model.to(device)

In [18]:
mobile_optim = optim.Adam(mobilenet_model.parameters(),lr=1e-03)
loss_fn = nn.CrossEntropyLoss()
epochs = 10

In [19]:
train_losses,test_losses = [],[]
for epoch in range(epochs):
    mobilenet_model.train()
    train_loss =0
    for batch,(x,y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        # print(x)
        pred = mobilenet_model(x)
        loss = loss_fn(pred,y)
        loss.backward()
        mobile_optim.step()
        mobile_optim.zero_grad()

        #Metrics 
        train_loss += loss.item()
        
    avg_loss = train_loss/len(train_loader)
    train_losses.append(avg_loss)
    print(f'Epoch: {epoch} | Train loss: {avg_loss}')
    # scheduler.step()

    mobilenet_model.eval()
    with torch.no_grad():
        test_loss,correct=0,0
        total_samples = 0
        for batch,(x,y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            pred = mobilenet_model(x)
            loss = loss_fn(pred,y)
            total_samples += y.size(0)
            test_loss += loss.item()
            correct += ((torch.argmax(torch.softmax(pred,dim=1),dim=1)==y).sum().item())
        avg_test_loss = test_loss/len(test_loader)
        avg_correct = correct/total_samples
        test_losses.append(avg_test_loss)
        print(f'Test_loss {avg_test_loss} | Accuracy {avg_correct}')

Epoch: 0 | Train loss: 0.5086818202720274
Test_loss 0.3714613063130409 | Accuracy 0.878
Epoch: 1 | Train loss: 0.32924646226798787
Test_loss 0.35196752029999046 | Accuracy 0.8803
Epoch: 2 | Train loss: 0.2773449925129371
Test_loss 0.3356719075399599 | Accuracy 0.8837
Epoch: 3 | Train loss: 0.2364119723334413
Test_loss 0.29346024107401536 | Accuracy 0.9014
Epoch: 4 | Train loss: 0.2036798616914112
Test_loss 0.30990770070036505 | Accuracy 0.901
Epoch: 5 | Train loss: 0.18671543073252111
Test_loss 0.3025070992624684 | Accuracy 0.9073
Epoch: 6 | Train loss: 0.16246559406104294
Test_loss 0.33433228228122563 | Accuracy 0.8999
Epoch: 7 | Train loss: 0.1525526876558962
Test_loss 0.2844889511111056 | Accuracy 0.9124
Epoch: 8 | Train loss: 0.13239426313139632
Test_loss 0.2722835362574477 | Accuracy 0.9156
Epoch: 9 | Train loss: 0.12748601459695594
Test_loss 0.3002992593653642 | Accuracy 0.9118


Mobilenet v3

In [9]:
# Define transformations
train_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
test_transform = transforms.Compose([
    transforms.Resize((224,224)),#usng this for AlexNet
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create PyTorch datasets and loaders
train_dataset = CIFAR10Dataset(train_data, train_labels, transform=test_transform)
test_dataset = CIFAR10Dataset(test_data, test_labels, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True,num_workers = 4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,num_workers = 4)

In [11]:
mobilev3 = models.mobilenet_v3_large(pretrained = True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:01<00:00, 11.5MB/s]


In [14]:
mobilev3.classifier[3] = nn.Linear(mobilev3.classifier[3].in_features, 10)
mobilev3 = mobilev3.to(device)


In [15]:
mobile_optim_v2 = optim.Adam(mobilev3.parameters(),lr=1e-03)
loss_fn = nn.CrossEntropyLoss()
epochs = 10

In [17]:
train_losses,test_losses = [],[]
for epoch in range(epochs):
    mobilev3.train()
    train_loss =0
    for batch,(x,y) in enumerate(train_loader):
        x = x.to(device)
        y = y.to(device)
        # print(x)
        pred = mobilev3(x)
        loss = loss_fn(pred,y)
        loss.backward()
        mobile_optim_v2.step()
        mobile_optim_v2.zero_grad()

        #Metrics 
        train_loss += loss.item()
        
    avg_loss = train_loss/len(train_loader)
    train_losses.append(avg_loss)
    print(f'Epoch: {epoch} | Train loss: {avg_loss}')
    # scheduler.step()

    mobilev3.eval()
    with torch.no_grad():
        test_loss,correct=0,0
        total_samples = 0
        for batch,(x,y) in enumerate(test_loader):
            x = x.to(device)
            y = y.to(device)
            pred = mobilev3(x)
            loss = loss_fn(pred,y)
            total_samples += y.size(0)
            test_loss += loss.item()
            correct += ((torch.argmax(torch.softmax(pred,dim=1),dim=1)==y).sum().item())
        avg_test_loss = test_loss/len(test_loader)
        avg_correct = correct/total_samples
        test_losses.append(avg_test_loss)
        print(f'Test_loss {avg_test_loss} | Accuracy {avg_correct}')

Epoch: 0 | Train loss: 0.2048970817414391
Test_loss 0.3538882140615943 | Accuracy 0.8891
Epoch: 1 | Train loss: 0.1522601905810025
Test_loss 0.28603043623126234 | Accuracy 0.9075
Epoch: 2 | Train loss: 0.1305653397762038
Test_loss 0.3213075873958077 | Accuracy 0.9068
Epoch: 3 | Train loss: 0.11225873144531666
Test_loss 0.29218567572059523 | Accuracy 0.9193
Epoch: 4 | Train loss: 0.10155848245886143
Test_loss 0.23012143881267802 | Accuracy 0.9336
Epoch: 5 | Train loss: 0.09276268938306215
Test_loss 0.2707412792429043 | Accuracy 0.9249
Epoch: 6 | Train loss: 0.07984644481275097
Test_loss 0.27086054246611657 | Accuracy 0.9267
Epoch: 7 | Train loss: 0.07215666037726948
Test_loss 0.3043073424772852 | Accuracy 0.9239
Epoch: 8 | Train loss: 0.07305218773110814
Test_loss 0.36928580734570315 | Accuracy 0.907
Epoch: 9 | Train loss: 0.06704193004213459
Test_loss 0.2959562250810444 | Accuracy 0.9286
